# Задание по лекциям 2.

- Не нужно использовать эффективные вычислительные алгоритмы. Нужно использовать те алгоритмы, что были представлены на лекции или практических занятиях. 
- Остальное как обычно: за "похожие" решения всем задействованным 0 баллов; если используете решение из открытого источника — обязательно вставьте ссылку; не удаляйте формулировки; не выкладывайте в открытый источник.
- Можно использовать `numpy.array` для матриц и матричной арифметики и `numpy.linalg` для подсчёта ранга и определителя, для вычисления обратной матрицы, решения СЛУ и т.п. То есть то, что вы уже реализовывали в прошлом семестре, ещё раз реализовывать необязательно. Более того, можно использовать в любом из *заданий по лекциям* функции, реализованные ранее в других *заданиях по лекциям*. Если возникнут сомнения, можно ли использовать ту или иную функцию — лучше сразу поинтересуйтесь у меня.

$\mathbb{R}^n$ — вещественнозначное пространство вектор-**столбцов** со стандартным скалярным произведением.

**(1 балл) Задание 5.** Реализовать функцию, принимающую на вход набор координат $u = (u_1,\ldots,u_k),\ k\le n$ вектор-столбцов, и выдающую базис $$v = (v_1,\ldots,v_m),\ m \ge n-k,$$
ортогонального дополнения  для линейной оболочки данных векторов.

In [2]:
import numpy as np
import scipy.linalg as sla
from numpy import linalg as la
import math
from math import fabs

In [13]:
def orth_space(U):
  rank_U=la.matrix_rank(U)
  new_U=U[::,:rank_U:]
  b=np.zeros((len(new_U),1))
  Q,R =la.qr(new_U.T)
  for i in range (len(new_U.T)-1,len(new_U)):
    x_num=sla.solve(R[:,:len(new_U.T):],-1*R[:,i])
    x_num=np.insert(x_num,[len(x_num)],[1],axis=0)
    x=np.zeros((len(new_U)-len(new_U.T),len(new_U)))
    x[i-len(new_U.T)]=x_num  
  return x


U = np.array([[1,1,2],[2,1,3],[0,1,1]]).reshape((3,3))
# я просто в шоке эта фигня работает....
# в основе принципа лежит то, что до означивания фср, вектор из параметров можно представить в виде суммы
# векторов по каждой переменной, и еще вынести переменные как множители. Вот и фср
# Когда мы означиваем переменные, то рассматриваем случаи, что одна переменная равна 1, а остальные 0, в нашей реализации это все равно что
# все, кроме одного множителя равны 0, и существует только 1 вектор из той суммы, а это все равно что выразить решение только по столбцу 1 переменной

orth_space(U)

array([[-2.,  1.,  1.]])

**(1 балл) Задание 6.** Реализовать функцию, принимающую на вход набор координат $u = (u_1,\ldots,u_k),\ k\le n$ вектор-столбцов, и выдающую ортонормированный базис $$v = (v_1,\ldots,v_m),\ m \ge n-k,$$
для линейной оболочки данных векторов.

In [12]:
def orth_base(U):
  v= np.ones((len(U),len(U[0])))
  U=U.T
  v[0]=1/la.norm(U[0],ord =2)*U[0]
  for i in range(1,len(U)):
    v[i]= U[i]
    
    for j in range (0,i+1):

      v[i]-= (U[i].T@v[j]/la.norm(v[j],ord = 2 ))*v[j]

    v[i]=(1/la.norm(v[i],ord = 2 ))*v[i]

  return v

U=np.array([[1,1,1],[1,0,1],[0,1,1]])
orth_base(U)
  

array([[ 0.70710678,  0.70710678,  0.        ],
       [-0.40824829,  0.40824829, -0.81649658],
       [-0.57735027,  0.57735027,  0.57735027]])

**(1 балл) Задание 7.** Реализовать функцию, принимающую на вход набор координат $u = (u_1,\ldots,u_k),\ k\le n$ линейно независимых вектор-столбцов, и выдающую ортонормированный базис $$v = (v_1,\ldots,v_n)$$
всего пространства, такой что линейная оболочка векторов $(v_1,\ldots,v_k)$ совпадает с линейной оболочкой векторов $(u_1,\ldots,u_k)$.

In [14]:
def mega_orth_base(U):
  u1=orth_base(U)
  u2=orth_space(U)
  for i in range(0,len(u2)):
    u2[i]=1/la.norm(u2[i])*u2[i]
  u=np.hstack((u1,u2))
